In [1]:
import pathlib
import os
if not "changed_working_directory" in locals():
    changed_working_directory = True
    os.chdir(pathlib.Path().resolve().parent)
    
%load_ext autoreload
%autoreload 2

from tabGAN import TabGAN
from src import constants as const
import helpers
from v2_hp_tuning import fetch_hp_info

import pandas as pd
import numpy as np

imports_path = "src//imports.ipynb"

%run "$imports_path"

if const.dir.project() != os.getcwd():
    raise ValueError("Project directory not the same as in consts")

In [ ]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(
    categories=cats,  # Categories per feature
)       

In [ ]:
d1 = pd.DataFrame({"a": ["a", "b", "c", "a", "b"]})
d2 = pd.DataFrame({"a": ["a", "b", "c", "a", "e"]})

In [ ]:
ohe.fit(d1)

In [ ]:
ohe.fit(d1.append(d2, ignore_index=True))

In [ ]:
ohe.categories_

In [ ]:
ohe.transform(d2)

In [ ]:
cats = ohe.categories_

In [ ]:
df = pd.read_csv(os.path.join(const.dir.data_comparison(), "news_edited.csv"))

In [ ]:
df.columns

In [ ]:
a = pd.Series(["d", "a", "a", "c", "b", "c"]).value_counts(sort=False)

In [6]:
def get_categories_or_range_from_df(df, round_decimals=None, round_significant_figures=2,
                                   round_decimals_numeric=2):
    df = df[sorted(df.columns, key=str.casefold)]
    #print(df.dtypes)
    df_overview_columns = pd.DataFrame({"Column": df.columns, "Values (numerical range or categories)": None})
    column_dtypes = df.dtypes
    for i, column in enumerate(df.columns):
        if column_dtypes[i] == "object":
            unique_categories = np.sort(df[column].unique())
            counts_categories = df[column].value_counts()
            percentage_categories = (counts_categories / counts_categories.sum() * 100)
            if round_significant_figures is not None:
                percentage_categories = percentage_categories.apply(lambda val: float('%.1g' % val))
                percentage_categories = percentage_categories.apply(lambda val: int(val) if val % 1 == 0 else val)
            elif round_decimals is not None:
                percentage_categories = percentage_categories.round(decimals=round_decimals)
            dict_category_percentages = {ix: val for ix,val in zip(percentage_categories.index, percentage_categories)}
            str_unique_categories = ", ".join(f"{cat} ({dict_category_percentages[cat]}%)"for cat in unique_categories)
            df_overview_columns.loc[i, "Values (numerical range or categories)"] = str_unique_categories
    #         print(str_unique_categories)
        else:
            low = df[column].min().round(round_decimals_numeric)
            high = df[column].max().round(round_decimals_numeric)
            df_overview_columns.loc[i, "Values (numerical range or categories)"] = f"[{low}, {high}]"
    #         print(f"[{df[column].min()}, {df[column].max()}]")
    return df_overview_columns

In [ ]:
get_categories_or_range_from_df(df)

In [4]:
def create_caption_for_column_overview_table(dataset_name):
    return (f"Overview of each column in the {dataset_name} dataset. For numerical columns the value range is reported,"
            " while for discrete columns the categories along with percentages are reported. The percentages for all"
            " categories of a single discrete column sum to $1$")

In [46]:
df = pd.read_csv(os.path.join(const.dir.data_comparison(), "covtype_edited.csv"))
from tabGAN import TabGAN
tg = TabGAN(df, quantile_transformation_int=True, print_values_where_qtr_is_applied=True, qtr_lbound_apply=0.05)

Column: Slope. Value: 8.0. n_curr_references: 51
Column: Slope. Value: 9.0. n_curr_references: 55
Column: Slope. Value: 10.0. n_curr_references: 57
Column: Slope. Value: 11.0. n_curr_references: 59
Column: Slope. Value: 12.0. n_curr_references: 57
Column: Slope. Value: 13.0. n_curr_references: 57
Column: Slope. Value: 14.0. n_curr_references: 53
Column: Horizontal_Distance_To_Hydrology. Value: 30.0. n_curr_references: 59
Column: Vertical_Distance_To_Hydrology. Value: 0.0. n_curr_references: 66


In [39]:
df.dtypes

Wilderness_Area                       object
Soil_Type                             object
Elevation                              int64
Aspect                                 int64
Slope                                  int64
Horizontal_Distance_To_Hydrology       int64
Vertical_Distance_To_Hydrology         int64
Horizontal_Distance_To_Roadways        int64
Hillshade_9am                          int64
Hillshade_Noon                         int64
Hillshade_3pm                          int64
Horizontal_Distance_To_Fire_Points     int64
Cover_Type                            object
dtype: object

In [18]:
df["Cover_Type"].value_counts()

CoverType2    283301
CoverType1    211840
CoverType3     35754
CoverType7     20510
CoverType6     17367
CoverType5      9493
CoverType4      2747
Name: Cover_Type, dtype: int64

In [57]:
df = pd.read_csv(os.path.join(const.dir.data_comparison(), "creditcard_edited.csv"))
from tabGAN import TabGAN
tg = TabGAN(df, quantile_transformation_int=True, quantile_rand_transformation=True, print_values_where_qtr_is_applied=True,
           qtr_lbound_apply=0)

In [41]:
df.dtypes

Time      float64
V1        float64
V2        float64
V3        float64
V4        float64
V5        float64
V6        float64
V7        float64
V8        float64
V9        float64
V10       float64
V11       float64
V12       float64
V13       float64
V14       float64
V15       float64
V16       float64
V17       float64
V18       float64
V19       float64
V20       float64
V21       float64
V22       float64
V23       float64
V24       float64
V25       float64
V26       float64
V27       float64
V28       float64
Amount    float64
Class      object
dtype: object

In [20]:
df["Class"].value_counts()

Class0    284315
Class1       492
Name: Class, dtype: int64

In [54]:
df = pd.read_csv(os.path.join(const.dir.data_comparison(), "creditcard_edited.csv"))
from tabGAN import TabGAN
tg = TabGAN(df, quantile_transformation_int=True, print_values_where_qtr_is_applied=True, qtr_lbound_apply=0)

In [ ]:
df_column_info_covtype = get_categories_or_range_from_df(pd.read_csv(os.path.join(const.dir.data_comparison(), "covtype_edited.csv")))
dataset_name = "Covertype"
with pd.option_context("max_colwidth", 1000):
    print(df_column_info_covtype.to_latex(caption=create_caption_for_column_overview_table(dataset_name),
                                     label=f"tab:column_overview_{dataset_name}",
                                      index=False))

In [ ]:
df_column_info_creditcard = get_categories_or_range_from_df(pd.read_csv(os.path.join(const.dir.data_comparison(), "creditcard_edited.csv")),)
dataset_name = "Creditcard"
print(df_column_info_creditcard.to_latex(caption=create_caption_for_column_overview_table(dataset_name),
                                     label=f"tab:column_overview_{dataset_name}",
                                      index=False))

In [8]:
df_column_info_news = get_categories_or_range_from_df(pd.read_csv(os.path.join(const.dir.data_comparison(), "news_edited.csv")))
dataset_name = "Online News Popularity"
with pd.option_context("max_colwidth", 200):
    print(df_column_info_news.to_latex(caption=create_caption_for_column_overview_table(dataset_name),
                                     label=f"tab:column_overview_{dataset_name}",
                                      index=False))

\begin{table}
\centering
\caption{Overview of each column in the Online News Popularity dataset. For numerical columns the value range is reported, while for discrete columns the categories along with percentages are reported. The percentages for all categories of a single discrete column sum to $1$}
\label{tab:column_overview_Online News Popularity}
\begin{tabular}{ll}
\toprule
                      Column &                                                                 Values (numerical range or categories) \\
\midrule
abs\_title\_sentiment\_polarity &                                                                                             [0.0, 1.0] \\
      abs\_title\_subjectivity &                                                                                             [0.0, 0.5] \\
        average\_token\_length &                                                                                            [0.0, 8.04] \\
       avg\_negative\_polarity &                      

In [ ]:
intrusion = pd.read_csv(os.path.join(const.dir.data_comparison(), "kddcup (3).data"),
                names=['duration', 'protocol_type', 'service', 'flag', 'src_bytes',
       'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot',
       'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell',
       'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
       'num_access_files', 'num_outbound_cmds', 'is_host_login',
       'is_guest_login', 'count', 'srv_count', 'serror_rate',
       'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
       'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
       'dst_host_srv_count', 'dst_host_same_srv_rate',
       'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
       'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
       'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
       'dst_host_srv_rerror_rate', 'class'])

In [ ]:
intrusion_edited = intrusion
intrusion_edited["logged_in"] = np.where(intrusion["logged_in"], "Yes", "No")
intrusion_edited["is_guest_login"] = np.where(intrusion["is_guest_login"], "Yes", "No")
intrusion_edited["land"] = np.where(intrusion["land"], "Yes", "No")
intrusion_edited["root_shell"] = np.where(intrusion["root_shell"], "Yes", "No")
intrusion_edited["intrusion"] = np.where(intrusion["class"] == "normal.", "No", "Yes")
intrusion_edited = intrusion_edited.drop("class", axis=1)
intrusion_edited = intrusion.dropna()
intrusion_edited = intrusion_edited.iloc[np.random.choice(np.arange(intrusion_edited.shape[0]), size=300000, replace=False),
                                         :]
intrusion_edited.to_csv(os.path.join(const.dir.data_comparison(), "intrusion_edited.csv"), index=False)

In [ ]:
for column, ty in zip(intrusion_edited.columns, intrusion_edited.dtypes):
    print(column, ty, intrusion_edited[column].unique())